In [2]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image

#creating the dataset class, the template can be seen in:
#https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

class BCEDetectDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path).convert("RGB")

        mask = np.array(mask)
        
        mask_map = mask[:,:,0]
        labels = []
        obj_ids = np.unique(mask[:,:,0])
        obj_ids = obj_ids[1:]
        for each_unique in obj_ids:
            x,y = np.where(mask[:,:,0]==each_unique)
            labels.append(mask[x[0],y[0],1])
        labels = torch.tensor(labels,dtype = torch.int64)

        masks = mask_map == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
#         labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [4]:
# this is part of the tutorial, which is needed here:
# #
# import torchvision
# from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
 
# # load a model pre-trained pre-trained on COCO
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
 
# # replace the classifier with a new one, that has
# # num_classes which is user-defined
# num_classes = 3  # 2 class (text/non-text)  + background
# # get number of input features for the classifier
# in_features = model.roi_heads.box_predictor.cls_score.in_features
# # replace the pre-trained head with a new one
# model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 

In [5]:
# this is part of the tutorial, which is needed here:
# import torchvision
# from torchvision.models.detection import FasterRCNN
# from torchvision.models.detection.rpn import AnchorGenerator
 
# # load a pre-trained model for classification and return
# # only the features
# backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# # FasterRCNN needs to know the number of
# # output channels in a backbone. For mobilenet_v2, it's 1280
# # so we need to add it here
# backbone.out_channels = 1280
 
# # let's make the RPN generate 5 x 3 anchors per spatial
# # location, with 5 different sizes and 3 different aspect
# # ratios. We have a Tuple[Tuple[int]] because each feature
# # map could potentially have different sizes and
# # aspect ratios 
# anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
#                                    aspect_ratios=((0.5, 1.0, 2.0),))
 
# # let's define what are the feature maps that we will
# # use to perform the region of interest cropping, as well as
# # the size of the crop after rescaling.
# # if your backbone returns a Tensor, featmap_names is expected to
# # be [0]. More generally, the backbone should return an
# # OrderedDict[Tensor], and in featmap_names you can choose which
# # feature maps to use.
# roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
#                                                 output_size=7,
#                                                 sampling_ratio=2)
 
# # put the pieces together inside a FasterRCNN model
# model = FasterRCNN(backbone,
#                    num_classes=3,
#                    rpn_anchor_generator=anchor_generator,
#                    box_roi_pool=roi_pooler)

In [6]:
#create the Mask-RCNN model
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
     
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    model.box_nms_thresh = 0.5
    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [7]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T

#adjusting the input data format
def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [11]:
# use our dataset and defined transformations
dataset = BCEDetectDataset('training_set', get_transform(train=True))
dataset_test = BCEDetectDataset('training_set', get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)


In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has 3 classes - background, text and non-text
num_classes = 3

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [10]:
# let's train it for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    torch.save(model.state_dict(), "./current_model_nms_set_1")
    # evaluate on the test dataset
#     evaluate(model, data_loader_test, device=device)

Epoch: [0]  [   0/5975]  eta: 2:33:29  lr: 0.000010  loss: 4.0607 (4.0607)  loss_classifier: 0.8433 (0.8433)  loss_box_reg: 0.1196 (0.1196)  loss_mask: 2.7962 (2.7962)  loss_objectness: 0.2615 (0.2615)  loss_rpn_box_reg: 0.0402 (0.0402)  time: 1.5414  data: 0.6931  max mem: 2289
Epoch: [0]  [  10/5975]  eta: 0:39:28  lr: 0.000060  loss: 3.9236 (3.7420)  loss_classifier: 0.8052 (0.7701)  loss_box_reg: 0.1221 (0.1273)  loss_mask: 2.0225 (1.8697)  loss_objectness: 0.9238 (0.8600)  loss_rpn_box_reg: 0.0862 (0.1149)  time: 0.3971  data: 0.0756  max mem: 2736
Epoch: [0]  [  20/5975]  eta: 0:33:35  lr: 0.000110  loss: 2.2150 (2.7496)  loss_classifier: 0.5439 (0.5930)  loss_box_reg: 0.1001 (0.1102)  loss_mask: 1.0598 (1.4232)  loss_objectness: 0.2751 (0.5269)  loss_rpn_box_reg: 0.0701 (0.0962)  time: 0.2783  data: 0.0118  max mem: 2805
Epoch: [0]  [  30/5975]  eta: 0:31:58  lr: 0.000160  loss: 1.2913 (2.2610)  loss_classifier: 0.2574 (0.4696)  loss_box_reg: 0.0909 (0.1096)  loss_mask: 0.8130 (

Epoch: [0]  [ 300/5975]  eta: 0:28:17  lr: 0.001508  loss: 0.5095 (0.8620)  loss_classifier: 0.1220 (0.1974)  loss_box_reg: 0.0812 (0.1371)  loss_mask: 0.2082 (0.3607)  loss_objectness: 0.0348 (0.0813)  loss_rpn_box_reg: 0.0696 (0.0855)  time: 0.3011  data: 0.0153  max mem: 2946
Epoch: [0]  [ 310/5975]  eta: 0:28:14  lr: 0.001558  loss: 0.4775 (0.8485)  loss_classifier: 0.1127 (0.1942)  loss_box_reg: 0.0740 (0.1347)  loss_mask: 0.2082 (0.3554)  loss_objectness: 0.0253 (0.0795)  loss_rpn_box_reg: 0.0662 (0.0848)  time: 0.2961  data: 0.0142  max mem: 2946
Epoch: [0]  [ 320/5975]  eta: 0:28:09  lr: 0.001608  loss: 0.4282 (0.8377)  loss_classifier: 0.0932 (0.1915)  loss_box_reg: 0.0691 (0.1330)  loss_mask: 0.1857 (0.3506)  loss_objectness: 0.0227 (0.0783)  loss_rpn_box_reg: 0.0446 (0.0843)  time: 0.2940  data: 0.0157  max mem: 2946
Epoch: [0]  [ 330/5975]  eta: 0:28:07  lr: 0.001658  loss: 0.4341 (0.8289)  loss_classifier: 0.1002 (0.1894)  loss_box_reg: 0.0712 (0.1319)  loss_mask: 0.1940 (

Epoch: [0]  [ 600/5975]  eta: 0:27:06  lr: 0.003007  loss: 0.4581 (0.6885)  loss_classifier: 0.1150 (0.1594)  loss_box_reg: 0.0883 (0.1119)  loss_mask: 0.1664 (0.2806)  loss_objectness: 0.0091 (0.0558)  loss_rpn_box_reg: 0.0476 (0.0808)  time: 0.2985  data: 0.0108  max mem: 2971
Epoch: [0]  [ 610/5975]  eta: 0:27:02  lr: 0.003057  loss: 0.3871 (0.6838)  loss_classifier: 0.1001 (0.1583)  loss_box_reg: 0.0772 (0.1111)  loss_mask: 0.1687 (0.2792)  loss_objectness: 0.0117 (0.0551)  loss_rpn_box_reg: 0.0348 (0.0801)  time: 0.2921  data: 0.0096  max mem: 2971
Epoch: [0]  [ 620/5975]  eta: 0:26:59  lr: 0.003107  loss: 0.3985 (0.6810)  loss_classifier: 0.0865 (0.1575)  loss_box_reg: 0.0739 (0.1107)  loss_mask: 0.1888 (0.2780)  loss_objectness: 0.0137 (0.0547)  loss_rpn_box_reg: 0.0332 (0.0801)  time: 0.2971  data: 0.0112  max mem: 2971
Epoch: [0]  [ 630/5975]  eta: 0:26:56  lr: 0.003157  loss: 0.4554 (0.6771)  loss_classifier: 0.1006 (0.1565)  loss_box_reg: 0.0756 (0.1102)  loss_mask: 0.1888 (

Epoch: [0]  [ 900/5975]  eta: 0:25:38  lr: 0.004505  loss: 0.4405 (0.6132)  loss_classifier: 0.0978 (0.1420)  loss_box_reg: 0.0658 (0.0997)  loss_mask: 0.1774 (0.2491)  loss_objectness: 0.0231 (0.0456)  loss_rpn_box_reg: 0.0558 (0.0768)  time: 0.3087  data: 0.0178  max mem: 3009
Epoch: [0]  [ 910/5975]  eta: 0:25:36  lr: 0.004555  loss: 0.4790 (0.6120)  loss_classifier: 0.1002 (0.1416)  loss_box_reg: 0.0807 (0.0995)  loss_mask: 0.1811 (0.2483)  loss_objectness: 0.0232 (0.0454)  loss_rpn_box_reg: 0.0775 (0.0772)  time: 0.3082  data: 0.0133  max mem: 3009
Epoch: [0]  [ 920/5975]  eta: 0:25:33  lr: 0.004605  loss: 0.4568 (0.6099)  loss_classifier: 0.1002 (0.1412)  loss_box_reg: 0.0798 (0.0992)  loss_mask: 0.1811 (0.2475)  loss_objectness: 0.0208 (0.0451)  loss_rpn_box_reg: 0.0525 (0.0768)  time: 0.3127  data: 0.0146  max mem: 3075
Epoch: [0]  [ 930/5975]  eta: 0:25:31  lr: 0.004655  loss: 0.4064 (0.6079)  loss_classifier: 0.0949 (0.1408)  loss_box_reg: 0.0713 (0.0990)  loss_mask: 0.1670 (

Epoch: [0]  [1200/5975]  eta: 0:24:17  lr: 0.005000  loss: 0.4182 (0.5755)  loss_classifier: 0.0869 (0.1325)  loss_box_reg: 0.0597 (0.0930)  loss_mask: 0.1644 (0.2330)  loss_objectness: 0.0198 (0.0406)  loss_rpn_box_reg: 0.0533 (0.0764)  time: 0.2997  data: 0.0124  max mem: 3075
Epoch: [0]  [1210/5975]  eta: 0:24:14  lr: 0.005000  loss: 0.3860 (0.5742)  loss_classifier: 0.0806 (0.1320)  loss_box_reg: 0.0590 (0.0927)  loss_mask: 0.1692 (0.2325)  loss_objectness: 0.0180 (0.0405)  loss_rpn_box_reg: 0.0542 (0.0765)  time: 0.3004  data: 0.0116  max mem: 3075
Epoch: [0]  [1220/5975]  eta: 0:24:10  lr: 0.005000  loss: 0.3954 (0.5734)  loss_classifier: 0.0917 (0.1320)  loss_box_reg: 0.0721 (0.0927)  loss_mask: 0.1638 (0.2320)  loss_objectness: 0.0210 (0.0403)  loss_rpn_box_reg: 0.0353 (0.0763)  time: 0.3021  data: 0.0118  max mem: 3075
Epoch: [0]  [1230/5975]  eta: 0:24:07  lr: 0.005000  loss: 0.3918 (0.5721)  loss_classifier: 0.0934 (0.1318)  loss_box_reg: 0.0787 (0.0925)  loss_mask: 0.1643 (

...

Epoch: [9]  [4590/5975]  eta: 0:07:13  lr: 0.000005  loss: 0.0965 (0.1038)  loss_classifier: 0.0197 (0.0216)  loss_box_reg: 0.0070 (0.0075)  loss_mask: 0.0551 (0.0550)  loss_objectness: 0.0008 (0.0020)  loss_rpn_box_reg: 0.0122 (0.0176)  time: 0.3046  data: 0.0109  max mem: 3233
Epoch: [9]  [4600/5975]  eta: 0:07:10  lr: 0.000005  loss: 0.0953 (0.1038)  loss_classifier: 0.0234 (0.0216)  loss_box_reg: 0.0056 (0.0075)  loss_mask: 0.0531 (0.0550)  loss_objectness: 0.0008 (0.0020)  loss_rpn_box_reg: 0.0111 (0.0176)  time: 0.3037  data: 0.0112  max mem: 3233
Epoch: [9]  [4610/5975]  eta: 0:07:07  lr: 0.000005  loss: 0.0957 (0.1037)  loss_classifier: 0.0207 (0.0216)  loss_box_reg: 0.0066 (0.0075)  loss_mask: 0.0515 (0.0550)  loss_objectness: 0.0011 (0.0020)  loss_rpn_box_reg: 0.0134 (0.0176)  time: 0.3099  data: 0.0139  max mem: 3233
Epoch: [9]  [4620/5975]  eta: 0:07:04  lr: 0.000005  loss: 0.0957 (0.1038)  loss_classifier: 0.0192 (0.0216)  loss_box_reg: 0.0067 (0.0075)  loss_mask: 0.0506 (

Epoch: [9]  [4890/5975]  eta: 0:05:40  lr: 0.000005  loss: 0.1059 (0.1039)  loss_classifier: 0.0210 (0.0216)  loss_box_reg: 0.0073 (0.0075)  loss_mask: 0.0528 (0.0550)  loss_objectness: 0.0009 (0.0020)  loss_rpn_box_reg: 0.0191 (0.0177)  time: 0.3088  data: 0.0125  max mem: 3233
Epoch: [9]  [4900/5975]  eta: 0:05:37  lr: 0.000005  loss: 0.0988 (0.1039)  loss_classifier: 0.0196 (0.0216)  loss_box_reg: 0.0076 (0.0075)  loss_mask: 0.0528 (0.0550)  loss_objectness: 0.0009 (0.0020)  loss_rpn_box_reg: 0.0191 (0.0177)  time: 0.3614  data: 0.0621  max mem: 3233
Epoch: [9]  [4910/5975]  eta: 0:05:34  lr: 0.000005  loss: 0.0946 (0.1039)  loss_classifier: 0.0216 (0.0216)  loss_box_reg: 0.0076 (0.0075)  loss_mask: 0.0535 (0.0550)  loss_objectness: 0.0011 (0.0020)  loss_rpn_box_reg: 0.0120 (0.0177)  time: 0.3894  data: 0.0923  max mem: 3233
Epoch: [9]  [4920/5975]  eta: 0:05:31  lr: 0.000005  loss: 0.1064 (0.1039)  loss_classifier: 0.0213 (0.0216)  loss_box_reg: 0.0061 (0.0075)  loss_mask: 0.0558 (

Epoch: [9]  [5190/5975]  eta: 0:04:06  lr: 0.000005  loss: 0.0976 (0.1038)  loss_classifier: 0.0225 (0.0216)  loss_box_reg: 0.0069 (0.0075)  loss_mask: 0.0544 (0.0550)  loss_objectness: 0.0006 (0.0020)  loss_rpn_box_reg: 0.0100 (0.0177)  time: 0.3121  data: 0.0147  max mem: 3233
Epoch: [9]  [5200/5975]  eta: 0:04:03  lr: 0.000005  loss: 0.0918 (0.1038)  loss_classifier: 0.0211 (0.0216)  loss_box_reg: 0.0066 (0.0075)  loss_mask: 0.0535 (0.0550)  loss_objectness: 0.0009 (0.0020)  loss_rpn_box_reg: 0.0100 (0.0177)  time: 0.3028  data: 0.0114  max mem: 3233
Epoch: [9]  [5210/5975]  eta: 0:03:59  lr: 0.000005  loss: 0.0974 (0.1038)  loss_classifier: 0.0211 (0.0216)  loss_box_reg: 0.0054 (0.0075)  loss_mask: 0.0537 (0.0550)  loss_objectness: 0.0008 (0.0020)  loss_rpn_box_reg: 0.0105 (0.0177)  time: 0.3042  data: 0.0114  max mem: 3233
Epoch: [9]  [5220/5975]  eta: 0:03:56  lr: 0.000005  loss: 0.0984 (0.1038)  loss_classifier: 0.0198 (0.0216)  loss_box_reg: 0.0065 (0.0075)  loss_mask: 0.0537 (

Epoch: [9]  [5490/5975]  eta: 0:02:32  lr: 0.000005  loss: 0.1034 (0.1038)  loss_classifier: 0.0220 (0.0216)  loss_box_reg: 0.0075 (0.0075)  loss_mask: 0.0541 (0.0550)  loss_objectness: 0.0015 (0.0020)  loss_rpn_box_reg: 0.0172 (0.0177)  time: 0.3646  data: 0.0649  max mem: 3233
Epoch: [9]  [5500/5975]  eta: 0:02:29  lr: 0.000005  loss: 0.0975 (0.1038)  loss_classifier: 0.0220 (0.0216)  loss_box_reg: 0.0071 (0.0075)  loss_mask: 0.0571 (0.0550)  loss_objectness: 0.0011 (0.0020)  loss_rpn_box_reg: 0.0113 (0.0177)  time: 0.3655  data: 0.0637  max mem: 3233
Epoch: [9]  [5510/5975]  eta: 0:02:25  lr: 0.000005  loss: 0.0916 (0.1038)  loss_classifier: 0.0204 (0.0216)  loss_box_reg: 0.0061 (0.0075)  loss_mask: 0.0539 (0.0550)  loss_objectness: 0.0006 (0.0020)  loss_rpn_box_reg: 0.0113 (0.0177)  time: 0.3096  data: 0.0114  max mem: 3233
Epoch: [9]  [5520/5975]  eta: 0:02:22  lr: 0.000005  loss: 0.0966 (0.1038)  loss_classifier: 0.0194 (0.0216)  loss_box_reg: 0.0060 (0.0075)  loss_mask: 0.0519 (

Epoch: [9]  [5790/5975]  eta: 0:00:58  lr: 0.000005  loss: 0.0952 (0.1038)  loss_classifier: 0.0178 (0.0216)  loss_box_reg: 0.0058 (0.0075)  loss_mask: 0.0533 (0.0550)  loss_objectness: 0.0010 (0.0020)  loss_rpn_box_reg: 0.0144 (0.0177)  time: 0.3039  data: 0.0116  max mem: 3233
Epoch: [9]  [5800/5975]  eta: 0:00:54  lr: 0.000005  loss: 0.0990 (0.1038)  loss_classifier: 0.0170 (0.0216)  loss_box_reg: 0.0059 (0.0075)  loss_mask: 0.0538 (0.0550)  loss_objectness: 0.0011 (0.0020)  loss_rpn_box_reg: 0.0139 (0.0177)  time: 0.3039  data: 0.0112  max mem: 3233
Epoch: [9]  [5810/5975]  eta: 0:00:51  lr: 0.000005  loss: 0.1010 (0.1038)  loss_classifier: 0.0178 (0.0216)  loss_box_reg: 0.0069 (0.0075)  loss_mask: 0.0504 (0.0550)  loss_objectness: 0.0011 (0.0020)  loss_rpn_box_reg: 0.0139 (0.0177)  time: 0.3033  data: 0.0135  max mem: 3233
Epoch: [9]  [5820/5975]  eta: 0:00:48  lr: 0.000005  loss: 0.0958 (0.1038)  loss_classifier: 0.0215 (0.0216)  loss_box_reg: 0.0065 (0.0075)  loss_mask: 0.0512 (

In [12]:
# pick one image from the test set
img, _ = dataset_test[48]
# put the model in evaluation mode 
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])